<a href="https://colab.research.google.com/github/cbellini153/Colab/blob/main/Conversi%C3%B3n_Ev_lineales_en_Ev_Paradas_Procesos_Batch.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import system
import datetime


#first = system.date.now()
#second = system.date.addMinutes(first, 119)

#print "segundos entre las 2 fechas", system.date.secondsBetween(first, second)

equipo='[default]HMI_PlantaETAC/'  #/NroPantalla
TablaEv=system.tag.read(str(equipo) + "TablaEventos").value
tabla=TablaEv
print 'TablaEv'
print TablaEv
horaact=system.tag.read("[default]Compartidos/FechaHoraIgnition").value
#ID_GP_FINAL_ORDEN=0
ID_P=0
ULT_ID_P_ev=0
ULT_ID_GP_ev=0

Resultado33 = system.db.runPrepQuery("""
	SELECT top 1 ID_GP,ID_P FROM %s
	ORDER by Fecha desc, id desc
	""" %tabla,[],'PrincipalEnvasadorasSECTOR')
	#PisoPlantaSVR_SAP
	#WHERE ID_O=?  quito esto porque hoar vaa transferir eventos de parada ya sea que tenga o no ID_O ingresado.
for row in Resultado33:
	ULT_ID_GP_ev = row["ID_GP"]
	ULT_ID_P_ev = row["ID_P"]
print 'ULT_ID_GP_ev'
print ULT_ID_GP_ev
print 'ULT_ID_P_ev'
print ULT_ID_P_ev


tabla='ParadasOnlineElaboracion'
horaact=system.tag.read("[default]Compartidos/FechaHoraIgnition").value
recurso=system.tag.read(str(equipo) + "Recurso").value
ID_P=0
print '[ParadasOnlineElaboracion]'
#print ID_O
Resultado3 = system.db.runPrepQuery("""
	SELECT top 1 ID_GP FROM %s
	WHERE Recurso like ?
	ORDER by FechaInicioM desc
	""" %tabla,[recurso],'PrincipalSAP')
	#PisoPlantaSVR_SAP
	#WHERE ID_O=?  quito esto porque hoar vaa transferir eventos de parada ya sea que tenga o no ID_O ingresado.
for row in Resultado3:
	ID_GP_PO = row["ID_GP"]
print 'ID_GP_PO'
print ID_GP_PO



if len(Resultado3)>0:
	ID_GP_ult_transf=ID_GP_PO
	# CONSULTO LOS EVENTOS LINEALES ENTRE FI Y FF INGRESADO POR EL OPERADOR
	TablaEv=system.tag.read(str(equipo) + "TablaEventos").value
	Resultado2 = system.db.runPrepQuery("""
		SELECT TOP 300 ID, ID_P, ID_GP, Fecha, EstadoMaquina, CodigoParada, convert(varchar, Fecha, 8) AS Hora,
		TiempoMarcha, ProdNominal  FROM %s  WHERE  ID_GP>=?
		and  Fecha > DATEADD(hh,-360,GETDATE()) and ID_GP is not Null and ID_P is not Null and ID is not Null
		ORDER by Fecha ASC,ID asc
		 """%TablaEv,[int(ID_GP_ult_transf)],'PrincipalEnvasadorasSECTOR')
			#PisoPlantaSVR_SAP
			#and (EstadoMaquina='MARCHA' or EstadoMaquina='PARADA' or EstadoMaquina='DOSIFICA'  or EstadoMaquina='CODIGO')
			#Fecha >= convert(datetime, ?, 102) and Fecha <=convert(datetime, ?, 102) and  quito esta condicion porque ya no depende de la orden
	for row in Resultado2:
		ID = row["ID"]							#0
		ID_P=row["ID_P"]						#1
		ID_GP=row["ID_GP"]						#2
		Fecha= row["Fecha"]						#3
		EstadoMaquina=row["EstadoMaquina"]		#4
		CodigoParada = row["CodigoParada"]		#5
		Hora = row["Hora"]	                    #6
		TiempoMarcha = row["TiempoMarcha"]		#7
		ProdNominal = row["ProdNominal"]	    #8
else:
	ID_P_ult_transf=0
	# CONSULTO LOS EVENTOS LINEALES ENTRE FI Y FF INGRESADO POR EL OPERADOR
	TablaEv=system.tag.read(str(equipo) + "TablaEventos").value
	Resultado2 = system.db.runPrepQuery("""
		SELECT TOP 300 ID, ID_P, ID_GP, Fecha, EstadoMaquina, CodigoParada, convert(varchar, Fecha, 8) AS Hora,
		TiempoMarcha, ProdNominal FROM %s
		WHERE    Fecha > DATEADD(hh,-360,GETDATE()) and ID_GP is not Null and ID_P is not Null and ID is not Null
		ORDER by Fecha ASC,ID asc
		""" %TablaEv,[],'PrincipalEnvasadorasSECTOR')
			#PisoPlantaSVR_SAP
			#(EstadoMaquina='MARCHA' or EstadoMaquina='PARADA' or EstadoMaquina='DOSIFICA' or EstadoMaquina='CODIGO')
			#Fecha >= convert(datetime, ?, 102) and Fecha <=convert(datetime, ?, 102) and  quito esta condicion porque ya no depende de la orden
	for row in Resultado2:
		ID = row["ID"]							#0
		ID_P=row["ID_P"]						#1
		ID_GP=row["ID_GP"]						#2
		Fecha= row["Fecha"]						#3
		EstadoMaquina=row["EstadoMaquina"]		#4
		CodigoParada = row["CodigoParada"]		#5
		Hora = row["Hora"]		  				#6
		TiempoMarcha = row["TiempoMarcha"]		#7
		ProdNominal = row["ProdNominal"]	    #8

print 'ID_GP_ult_transf'
print ID_GP_ult_transf
print 'NRO DE EVENTOS CONSULTADOS: ',len(Resultado2)
if (len(Resultado2)>0):
	print 'Resultado2'
	print Resultado2
	array_res2=[[col for col in row] for row in Resultado2]
	print 'ultimo ID'
	print 	str(array_res2[(len(Resultado2)-1)][2])		  #len(Resultado2)-1
	print 	str(array_res2[(len(Resultado2)-1)][4])
	#-----------------------------------------------------------------------
	# RUTINA DE TRANSFORMACIÓN DE EV LINEALES A EV DE PARADAS

a=-1
ULT_IDGP = ID_GP_ult_transf
TieneEvPar=0

ULT_IDGP_completo= int(array_res2[(len(Resultado2)-1)][2])-1
print 'ultimo ID_GP completo que puedo transferir: ',	ULT_IDGP_completo


MotivoParada=""
MotivoParada_D=""
FechaInicioM=""
FechaFinM=""
FechaInicioP=""
FechaFinP=""
M=[]
Datos=0
FechaInicioP=('Null')
FechaFinP=('Null')
Mem=0
#print 'condición de mayor', len(Resultado2), 'ULT_ID_GP_ev',ULT_ID_GP_ev,'int(array_res2[(len(Resultado2)-1)][2])',int(array_res2[(len(Resultado2)-1)][2]),' - ID_GP_PO',ID_GP_PO
if len(Resultado2)>0 and ULT_ID_GP_ev>ID_GP_PO:      #int(array_res2[(len(Resultado2)-1)][2]):
	for i in range(len(Resultado2)) :  #len(Resultado2)   range(30)
		#ME POSICIONO EN EL ULTIMO ID_GP TRNASFERIDO Y PASO EL SIGUIENTE SI SE ENCUENTRA COMPLETO
		a=a+1
		print "array_res2[a][2] ",array_res2[a][2]
		print "ULT_IDGP",ULT_IDGP
		print "ULT_ID_GP_ev ",ULT_ID_GP_ev
		print "----------------"
		#print array_res2[a][2]

		if array_res2[a][2] > ULT_IDGP and ULT_IDGP<ULT_IDGP_completo:
			print "cond 1"
			if array_res2[a][7]==0 and array_res2[a][2] == (ULT_IDGP+1):
				print "cond 2"
				#TENGO LA FECHA DE INICIO DE MARCHA DEL CICIO SIGUIENTE
				FechaInicioM=(array_res2[a][3])
			if array_res2[a][7]==array_res2[a][8] and Mem==0: #significa que termina la marcha
				print "cond 3"
				FechaFinM=(array_res2[a][3])
				FechaInicioP=(array_res2[a][3])
				TieneEvPar=1
				Mem=1
			if array_res2[a+1][7]==0 and array_res2[a+1][2] == (ULT_IDGP+2):
				print "cond 4"
				ULT_IDGP=ULT_IDGP+1  #CIERRO EL CICLO ACTUAL

				if TieneEvPar==1:
					FechaFinP=(array_res2[a+1][3])
					#HACER INSERT

				else:
					FechaFinM=(array_res2[a+1][3])
					FechaInicioP=('')
					FechaFinP=('')
					#HACER INSERT
				TieneEvPar=0
				Mem=0
				#print "segundos entre las 2 fechas" + str(FechaInicioM) +" - "+ str(array_res2[a][3]), system.date.secondsBetween(FechaInicioM,array_res2[a][3])
				#TpoCiclo=0
				#TpoCiclo=system.date.secondsBetween(FechaInicioM,array_res2[a][3])
				#print "segundos entre las 2 fechas", system.date.secondsBetween(toDate("2017-04-28 00:00:00"), toDate("2017-03-22 00:00:00"))
				#print "segundos entre las 2 fechas", secondsBetween(toDate(FechaInicioM), toDate(array_res2[a][3]) )

				ULT_IDGP=array_res2[a][2]

				M.append(int(0)) 								#ID_O
				M.append(int(0)) 								#ID_GO
				M.append(int(array_res2[a][1])) 				#ID_P
				M.append(int(array_res2[a][2])) 				#ID_GP
				M.append(FechaInicioM)					        #Fecha Hora Ini  Mar
				M.append(FechaFinM)					       		#Fecha Hora Fin  Mar
				M.append("O6")								    #CP
				M.append(recurso)								#Recurso
				M.append("O6")									#CPoriginal
				M.append(int(990000))							#Visible
				M.append("")								    #CPD
				M.append(FechaInicioP)					        #Fecha Hora Ini  Par
				M.append(FechaFinP)					       		#Fecha Hora Fin  Par
				M.append(int(array_res2[a][8]))					#Tpo Ciclo

				Datos=Datos+1
				print M

	if Datos>0:
		tabla='ParadasOnlineElaboracion'
		I="insert into " + tabla
		#Insert+= tabla1
		I+=" (ID_O,ID_GO,ID_P,ID_GP,FechaInicioM,FechaFinM,CP,Recurso, CPoriginal, Visible,CPD,FechaInicioP,FechaFinP, TpoCiclo, FechaCreacion) values "   #?)"

		J=-1
		for i in range(Datos):
			J=J+1

			#z=z-1  #vuelvo al mismo z para que separe la parada en la misma hora

			if J==0:
				I+="( ?"  		#ID_O
			else:
				I+=",( ?" 		#ID_O
			I+=",?"       		#ID_GO
			I+=",?"       		#ID_P
			I+=",?"       		#ID_GP
			I+=",?"       		#FechaInicioM
			I+=",?"       		#FechaFinM
			I+=",?"       		#CP
			I+=",?"       		#Recurso
			I+=",?"       		#CPoriginal
			I+=",?"       		#Visible
			I+=",?"       		#CP
			I+=",?"       		#FechaInicioP
			I+=",?"       		#FechaFinP
			I+=",?"       		#TpoCiclo
			I+=",Getdate()"     #FechaCreacion
			I+=")"

		system.db.runPrepUpdate(""+I , M,"PrincipalSAP")
	else:
		print 'NO TENGO DATOS PARA ESCRIBIR'
